## TP3: Introducción al aprendizaje automático

In [38]:
import pandas as pd

In [39]:
Datos = pd.read_csv("https://raw.githubusercontent.com/Rondamon/2020_Mentoria_LluviasExtremas/master/dataset/datos_diarios_cordoba.csv", parse_dates = True, index_col="fecha")
id_Datos = pd.read_csv("https://raw.githubusercontent.com/Rondamon/2020_Mentoria_LluviasExtremas/master/dataset/metadatos_estaciones.csv")

De los datasets analizados en los TPs anteriores, ahora nos vamos a enfocar en:

- La Florida
- Marcos Juarez

Y vamos a usar como variable que queremos predecir, la PMDA que han calculado para cada año hidrológico (del 1/julio al 30/junio).

Las demás variables (como viento, temperatura, etc) las vamos a resumir para cada año hidrológico en la media (se pueden agregar más estadísticos pero así es suficiente).

##### Actividades:

##### OBJETIVO: Predecir Precipitación Máxima Diaria Anual (PMDA) para los próximos 5 años

1) De los dataset que han trabajado hasta ahora, agreguen más features al DataFrame **"Datos"** (Por lo general, más features tienden a dar mejores predicciones).

2) Además de las features del punto anterior, les propongo agregar nuevas features moviendo las originales tantos "pasos de tiempo" como features nuevas quieran generar.

3) Curar el dataset (si fuera necesario).

4) Aplicar técnicas de selección y extraccion de features. Armar un nuevo dataset con aquellas features más significativas.

5) Dividir el dataset (training, validation, test).. Recuerden que queremos predecir la PMDA para los próximos 5 años.

6) Analizar y elegir 3 modelos para REGRESIÓN. Entrenarlos y analizar resultados.

7) Evaluar predicciones de los diferentes modelos.

8) ¿Qué estadísticos utilizaron para evaluar las regresiones? Justifiquen su elección.

In [40]:
full_dataset = pd.read_csv('./datasetTP2/datos_diarios_cordoba_terra_chirps.csv')

In [41]:
dataset = full_dataset[(full_dataset['nombre'] == 'La Florida') | (full_dataset['nombre'] == 'Marcos Juarez INTA')]
dataset.loc[:, 'fecha'] = pd.to_datetime(dataset['fecha'])
dataset.reset_index(inplace=True)
dataset = dataset.drop(['Unnamed: 0'], axis=1)

/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [42]:
def get_hydro_year(date_, start_month=7):
    year = date_.strftime('%Y')
    month = date_.strftime('%m')
    offset = 1 if int(month) >= start_month - 1 else 0
    hydro_year = int(year) + offset - 1
    return hydro_year

def add_hydro_year(dataset):
    dataset = dataset.copy()
    dataset.reset_index(inplace=True)
    dataset['hydro_year'] = dataset['fecha'].apply(get_hydro_year)
    return dataset

def remove_cols(dataset, cols=['level_0', 'ano', 'mes', 'caudal', 'tmed', 'vmax_d', 'vmax_f']):
    return dataset.drop(cols, axis=1)

def get_grouped_dataset(dataset, group=['omm_id', 'hydro_year']):
    return dataset.groupby(group)

def derive_nth_day_feature(df, feature, N):
    rows = df.shape[0]
    nth_prior_measurements = [None]*N + [df[feature][i-N] for i in range(N, rows)]
    col_name = "{}_{}".format(feature, N)
    df[col_name] = nth_prior_measurements

In [43]:
dataset = add_hydro_year(dataset)
dataset = remove_cols(dataset)

In [44]:
dataset.tail()

,index,fecha,omm_id,nombre,prcp,tmax,tmin,vmed,ETreal_mm,HumedadSuelo_mm,RadSolar_Wpm2,PresionVapor_hPa,IndicePalmer_porc,hydro_year
41353,84826,2018-11-24,1,La Florida,0.0,24.4,10.5,5.38,114.5,68.0,258.1,12.26,1.29,2018
41354,84827,2018-11-25,1,La Florida,0.0,24.4,10.5,5.38,114.5,68.0,258.1,12.26,1.29,2018
41355,84828,2018-11-26,1,La Florida,21.5,24.4,10.5,5.38,114.5,68.0,258.1,12.26,1.29,2018
41356,84829,2018-11-27,1,La Florida,94.0,24.4,10.5,5.38,114.5,68.0,258.1,12.26,1.29,2018
41357,84830,2018-11-28,1,La Florida,0.0,24.4,10.5,5.38,114.5,68.0,258.1,12.26,1.29,2018


In [45]:
dataset.columns

Index(['index', 'fecha', 'omm_id', 'nombre', 'prcp', 'tmax', 'tmin', 'vmed',
       'ETreal_mm', 'HumedadSuelo_mm', 'RadSolar_Wpm2', 'PresionVapor_hPa',
       'IndicePalmer_porc', 'hydro_year'],
      dtype='object')

In [46]:
features_columns = ['tmax', 'tmin', 'vmed',
       'ETreal_mm', 'HumedadSuelo_mm', 'RadSolar_Wpm2', 'PresionVapor_hPa',
       'IndicePalmer_porc']
target = ['prcp']

In [47]:
for feature in features_columns:
    if feature != 'prcp':
        for N in range(1, 4):
            derive_nth_day_feature(dataset, feature, N)

In [48]:
print(dataset.isnull().values.any())

True


In [49]:
dataset = dataset.dropna()

In [50]:
dataset.columns

Index(['index', 'fecha', 'omm_id', 'nombre', 'prcp', 'tmax', 'tmin', 'vmed',
       'ETreal_mm', 'HumedadSuelo_mm', 'RadSolar_Wpm2', 'PresionVapor_hPa',
       'IndicePalmer_porc', 'hydro_year', 'tmax_1', 'tmax_2', 'tmax_3',
       'tmin_1', 'tmin_2', 'tmin_3', 'vmed_1', 'vmed_2', 'vmed_3',
       'ETreal_mm_1', 'ETreal_mm_2', 'ETreal_mm_3', 'HumedadSuelo_mm_1',
       'HumedadSuelo_mm_2', 'HumedadSuelo_mm_3', 'RadSolar_Wpm2_1',
       'RadSolar_Wpm2_2', 'RadSolar_Wpm2_3', 'PresionVapor_hPa_1',
       'PresionVapor_hPa_2', 'PresionVapor_hPa_3', 'IndicePalmer_porc_1',
       'IndicePalmer_porc_2', 'IndicePalmer_porc_3'],
      dtype='object')

In [51]:
not_features_cols = ['index', 'fecha', 'omm_id', 'nombre', 'prcp']
features_columns = [col for col in dataset.columns if col not in not_features_cols]

In [52]:
features_dataset = dataset[features_columns]
target_df = dataset[target]
features_dataset.shape, target_df.shape

((39267, 33), (39267, 1))

In [53]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
import numpy as np

In [54]:
X = features_dataset.copy()
y = target_df.copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)
X_train.shape, X_test.shape

((31413, 33), (7854, 33))

In [55]:
# fig = plt.figure(figsize=(30,20))
# fig.subplots_adjust(hspace=0.5, wspace=0.5)

# i = 1
# for feature in features_columns:
#     ax = fig.add_subplot(3, 5, i)
#     plt.scatter(X.loc[:, feature], y, facecolor='dodgerblue', edgecolor='k', label='datos')
#     plt.title(feature)
#     i += 1
# plt.show()

In [69]:
feature = ['vmed', 'vmed_2']
# X_train_f = X_train.loc[:, feature]
# X_test_f = X_test.loc[:, feature]
X_train_f = X_train
X_test_f = X_test
X_train_f.shape, X_test_f.shape

((31413, 33), (7854, 33))

In [70]:
# model = LinearRegression()
# model.fit(X_train_f, y_train)

In [71]:
# y_train_pred = model.predict(X_train_f)
# y_test_pred = model.predict(X_test_f)

# train_error = mean_squared_error(y_train, y_train_pred)
# test_error = mean_squared_error(y_test, y_test_pred)
# print(f'Train error: {train_error}')
# print(f'Test error: {test_error}')

In [ ]:
# train:
model = make_pipeline(PolynomialFeatures(3), LinearRegression())
model.fit(X_train_f, y_train)

# predict:
y_train_pred = model.predict(X_train_f)
y_test_pred = model.predict(X_test_f)

# evaluate:
train_error = mean_squared_error(y_train, y_train_pred)
test_error = mean_squared_error(y_test, y_test_pred)

In [ ]:
train_error, test_error